## Data Preparation

In [1]:
import pandas as pd

In [2]:
movies_data_path = '../dataset/movies.csv'
finantial_data_path = '../dataset/finantials.csv'
opening_data_path = '../dataset/opening_gross.csv'

In [3]:
fin_data = pd.read_csv(finantial_data_path)
movie_data = pd.read_csv(movies_data_path)
opening_data = pd.read_csv(opening_data_path)

In [4]:
numeric_columns_mask = (movie_data.dtypes == float) | (movie_data.dtypes == int)
numeric_columns = [column for column in numeric_columns_mask.index if numeric_columns_mask[column]]
movie_data = movie_data[numeric_columns+['movie_title']]

In [5]:
fin_data = fin_data[['movie_title','production_budget','worldwide_gross']]

In [6]:
fin_movie_data = pd.merge(fin_data, movie_data, on= 'movie_title', how='left')

In [7]:
full_movie_data = pd.merge( opening_data,fin_movie_data, on = 'movie_title', how='left')

In [8]:
full_movie_data.head(3)

,movie_title,opening_gross,screens,production_budget,worldwide_gross,title_year,aspect_ratio,duration,cast_total_facebook_likes,budget,imdb_score,gross
0,10 Days in a Madhouse,2451.0,10.0,12000000,14616,2015.0,1.85,111.0,2059,12000000.0,7.5,14616.0
1,10 Things I Hate About You,8330681.0,2271.0,13000000,60414025,1999.0,1.85,97.0,37907,16000000.0,7.2,38176108.0
2,102 Dalmatians,19883351.0,2704.0,85000000,66941559,2000.0,1.85,100.0,4182,85000000.0,4.8,66941559.0


In [9]:
full_movie_data[(full_movie_data.worldwide_gross != 0) & (full_movie_data.worldwide_gross.notnull())].shape

(2304, 12)

In [10]:
full_movie_data = full_movie_data.drop(['movie_title','gross'],axis=1)

In [11]:
full_movie_data.head(3)

,opening_gross,screens,production_budget,worldwide_gross,title_year,aspect_ratio,duration,cast_total_facebook_likes,budget,imdb_score
0,2451.0,10.0,12000000,14616,2015.0,1.85,111.0,2059,12000000.0,7.5
1,8330681.0,2271.0,13000000,60414025,1999.0,1.85,97.0,37907,16000000.0,7.2
2,19883351.0,2704.0,85000000,66941559,2000.0,1.85,100.0,4182,85000000.0,4.8


In [12]:
full_movie_data.columns

Index(['opening_gross', 'screens', 'production_budget', 'worldwide_gross',
       'title_year', 'aspect_ratio', 'duration', 'cast_total_facebook_likes',
       'budget', 'imdb_score'],
      dtype='object')

## Modeling

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_validate
import numpy as np

In [14]:
X = full_movie_data.drop(['worldwide_gross'], axis = 1)
y = full_movie_data['worldwide_gross']
print(X.head(3))
print(y.head(3))

   opening_gross  screens  production_budget  title_year  aspect_ratio  \
0         2451.0     10.0           12000000      2015.0          1.85   
1      8330681.0   2271.0           13000000      1999.0          1.85   
2     19883351.0   2704.0           85000000      2000.0          1.85   

   duration  cast_total_facebook_likes      budget  imdb_score  
0     111.0                       2059  12000000.0         7.5  
1      97.0                      37907  16000000.0         7.2  
2     100.0                       4182  85000000.0         4.8  
0       14616
1    60414025
2    66941559
Name: worldwide_gross, dtype: int64


In [15]:
pipeline = Pipeline([
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('core_model', GradientBoostingRegressor())
])

In [16]:
results = cross_validate(pipeline ,X,y,return_train_score=True,cv=10)
results

{'fit_time': array([0.46714306, 0.33682537, 0.73785496, 0.71058083, 0.38228869,
        0.32263613, 0.35952425, 0.35263729, 0.57578039, 0.63139009]),
 'score_time': array([0.00244522, 0.00512767, 0.0041275 , 0.00387049, 0.00186682,
        0.00241518, 0.00331879, 0.00253749, 0.00402784, 0.00417829]),
 'test_score': array([0.67441052, 0.84738013, 0.64398525, 0.77568809, 0.78411151,
        0.85666934, 0.74977225, 0.87416125, 0.67502303, 0.65745399]),
 'train_score': array([0.91673951, 0.91581777, 0.9228721 , 0.91654412, 0.92172829,
        0.91476722, 0.92151444, 0.91734995, 0.92320705, 0.91766026])}

In [17]:
train_score = np.mean(results['train_score'])
test_score = np.mean(results['test_score'])
assert train_score > 0.7
assert test_score > 0.65
print(f'Train Score: {train_score}')
print(f'Test Score: {test_score}')

Train Score: 0.918820072167686
Test Score: 0.7538655352983242


## Hyperparameter tunning

In [18]:
from sklearn.model_selection import GridSearchCV

In [19]:
param_tunning = {'core_model__n_estimators': range(20,501,20)} 

In [20]:
estimator = Pipeline([
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('core_model', GradientBoostingRegressor())
])

In [21]:
grid_search= GridSearchCV(estimator,
                       param_grid = param_tunning,
                       scoring='r2',
                       cv=5) 

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.35,random_state= 42)

In [23]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('imputer', SimpleImputer()),
                                       ('core_model',
                                        GradientBoostingRegressor())]),
             param_grid={'core_model__n_estimators': range(20, 501, 20)},
             scoring='r2')

In [24]:
final_result = cross_validate(grid_search.best_estimator_,X_train,y_train,return_train_score=True,cv=7)

In [25]:
final_result

{'fit_time': array([0.88041496, 0.59635806, 0.49279523, 0.43736959, 0.74104238,
        0.86909652, 0.49441051]),
 'score_time': array([0.00712466, 0.00269365, 0.00248766, 0.003227  , 0.00500107,
        0.00491238, 0.00189018]),
 'test_score': array([0.60974634, 0.70452495, 0.88015729, 0.83829442, 0.76240343,
        0.79833828, 0.73010682]),
 'train_score': array([0.96553131, 0.96506988, 0.9644736 , 0.9678394 , 0.96825267,
        0.96958688, 0.96663831])}

In [26]:
train_score = np.mean(final_result['train_score'])
test_score = np.mean(final_result['test_score'])
assert train_score > 0.7
assert test_score > 0.65
print(f'Train Score: {train_score}')
print(f'Test Score: {test_score}')

Train Score: 0.9667702920281013
Test Score: 0.7605102162122618


In [27]:
grid_search.best_estimator_.get_params()

{'memory': None,
 'steps': [('imputer', SimpleImputer()),
  ('core_model', GradientBoostingRegressor(n_estimators=220))],
 'verbose': False,
 'imputer': SimpleImputer(),
 'core_model': GradientBoostingRegressor(n_estimators=220),
 'imputer__add_indicator': False,
 'imputer__copy': True,
 'imputer__fill_value': None,
 'imputer__missing_values': nan,
 'imputer__strategy': 'mean',
 'imputer__verbose': 'deprecated',
 'core_model__alpha': 0.9,
 'core_model__ccp_alpha': 0.0,
 'core_model__criterion': 'friedman_mse',
 'core_model__init': None,
 'core_model__learning_rate': 0.1,
 'core_model__loss': 'squared_error',
 'core_model__max_depth': 3,
 'core_model__max_features': None,
 'core_model__max_leaf_nodes': None,
 'core_model__min_impurity_decrease': 0.0,
 'core_model__min_samples_leaf': 1,
 'core_model__min_samples_split': 2,
 'core_model__min_weight_fraction_leaf': 0.0,
 'core_model__n_estimators': 220,
 'core_model__n_iter_no_change': None,
 'core_model__random_state': None,
 'core_model_

In [28]:
estimator = Pipeline([
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('core_model', GradientBoostingRegressor(n_estimators=220,
                                             alpha=0.9,
                                             ccp_alpha=0.0,
                                             criterion='friedman_mse',
                                             init=None,
                                             learning_rate=0.1,
                                             loss='squared_error',
                                             max_depth=3,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_iter_no_change=None,
                                             random_state=None,
                                             subsample=1.0,
                                             tol=0.0001,
                                             validation_fraction=0.1,
                                             verbose=0,
                                             warm_start=False))
])

In [29]:
estimator.fit(X_train,y_train)

Pipeline(steps=[('imputer', SimpleImputer()),
                ('core_model', GradientBoostingRegressor(n_estimators=220))])

In [30]:
estimator.score(X_test, y_test)

0.7312944190787165

## Saving model

In [31]:
from joblib import dump

In [34]:
dump(estimator, '../model/model.pkl')

['../model/model.pkl']

In [33]:
X_train.columns

Index(['opening_gross', 'screens', 'production_budget', 'title_year',
       'aspect_ratio', 'duration', 'cast_total_facebook_likes', 'budget',
       'imdb_score'],
      dtype='object')